In [1]:
from dqn_utils.models import *
from dqn_utils.utils import define_env, train, optimize_model, action_selection
from dqn_utils.models import manDQN
from adabyron_environment.AdaByronDEMO import AdaByronDEMO
from dqn_utils.memory import Memory
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import random
from collections import deque

In [2]:
K = 4
EPS_DECAY = 0.0000009
EPS_MIN = 0.1
BATCH_SIZE = 32
#BATCH_SIZE = 4
MAX_STEPS = 10000
STEP_UPDATE_MODEL = 5000
INPUT_SIZE = 1442
env, policy_network, target_network, action_space  = define_env("test", ["aire1", "aire2"], baterias=["bateria1"], INPUT_SIZE=INPUT_SIZE)

#model_type = "Dense" # Network (Dense)
#curve_pred = [35.0, 42.5]
#curve_pred = [35.0, 42.5, 47.5,  38.5, 33.0, 41.0, 46.0, 37.0]
curve_pred = [random.randint(30, 50) for i in range(1440)]

optimizer = tf.keras.optimizers.Adam(learning_rate=0.00025, clipnorm=1.0)
loss_function = tf.keras.losses.Huber()
seed = 42
gamma = 0.99
epsilon = 1
memory = Memory(10000) # Memory size

Loading models...


In [3]:
from dqn_utils.utils import action_selection
estados = []
initial_state = [0, 0]+curve_pred
for i in range(0, len(curve_pred)):
    tf_tensor = tf.convert_to_tensor(initial_state)
    tf_tensor = tf.expand_dims(initial_state, 0)

    output = policy_network(tf_tensor, training=False)
    action_mask = action_selection(output, action_space)
    print(action_mask)
    # qpvalues_mask = tf.one_hot([action_mask], action_space[0].n, axis=2).numpy()
    # qpvalues_mask = tf.reduce_sum(qpvalues_mask, axis=1).numpy()
    
    # qpvalues = tf.multiply(output, qpvalues_mask)
    # qpvalues = tf.reduce_sum(qpvalues, axis=1)
    
        
    next_step, reward, done = env.step(action_mask, i, curve_pred)
    
    estados.append(next_step)
    
    initial_state = [next_step, i+1]+curve_pred
# print(estados)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 